### Calculate cost by case of OW installation in NS

Considers 
* Inv. cost (Capex + Fixed O&M [Opex])
* Total cable cost in NS 
* Cost of hubs

In [3]:
import pandas as pd
from constants import OFFSHORE_AREAS, NO_NODES

pd.options.mode.chained_assignment = None  # default='warn'

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_INV_BY_CASES = dict({})
DF_TRANS_BY_CASES = dict({})
DF_HUBS_BY_CASES = dict({})

for case in CASES:
    _df_inv = pd.read_csv(f'Results/{case}/results_output_gen.csv')
    _df_inv = _df_inv[_df_inv["Node"].isin(OFFSHORE_AREAS)]
    _df_trans = pd.read_csv(f'Results/{case}/results_output_transmission.csv')
    _df_trans = _df_trans[(_df_trans["BetweenNode"].isin(OFFSHORE_AREAS)) | (_df_trans["AndNode"].isin(OFFSHORE_AREAS))]
    _df_hubs = pd.read_csv(f'Results/{case}/results_output_offshoreConverter.csv')

    DF_INV_BY_CASES[case] = _df_inv
    DF_TRANS_BY_CASES[case] = _df_trans
    DF_HUBS_BY_CASES[case] = _df_hubs

In [16]:
FILTER_CASE = "Base"
FILTER_PERIOD = "2045-2050"

PERIODS = [f"20{20+5*i}-20{25+5*i}" for i in range(8)]
in_year = FILTER_PERIOD.split("-")[1]
excl_periods = [p for p in PERIODS if p.split("-")[1] > in_year]

### INVESTMENT COST
df_inv = DF_INV_BY_CASES[FILTER_CASE]
df_inv = df_inv[~df_inv["Period"].isin(excl_periods)]

total_inst_cap_GW = round(df_inv[(df_inv["Period"] == FILTER_PERIOD)]["genInstalledCap_MW"].sum() / 1000, 1)
total_inv_cost_mEUR = round(df_inv["DiscountedInvestmentCost_Euro"].sum() / 10**6, 1)
inv_cost_mEURperGW = round(total_inv_cost_mEUR/total_inst_cap_GW, 1)

print(f"Total installed capacity NS {in_year}: {total_inst_cap_GW}GW")
print(f"Total investment cost NS {in_year}: {total_inv_cost_mEUR} mEUR (discount rate = 5%/yr)")
print(f"Total investment cost NS {in_year} per GW installed: {inv_cost_mEURperGW} mEUR (discount rate = 5%/yr)\n")

### TRANSMISSION COST (Cables excl. hub capacity extra cost)
df_trans = DF_TRANS_BY_CASES[FILTER_CASE]
df_trans = df_trans[~df_trans["Period"].isin(excl_periods)]

# Wrong unit from EMPIRE; Output in EUR not EUR/MW
df_trans["DiscountedInvestmentCost_Euro"] = df_trans["DiscountedInvestmentCost_EuroPerMW"]

total_inst_trans_cap_GW = round(df_trans[df_trans["Period"] == FILTER_PERIOD]["transmissionInstalledCap_MW"].sum() / 1000, 1)
total_trans_inv_cost_mEUR = round(df_trans["DiscountedInvestmentCost_Euro"].sum() / 10**6, 1)
trans_inv_cost_mEURperGW = round(total_trans_inv_cost_mEUR/total_inst_trans_cap_GW, 1)

print(f"Total installed trans. capacity NS {in_year}: {total_inst_trans_cap_GW}GW")
print(f"Total trans. investment cost NS {in_year}: {total_trans_inv_cost_mEUR} mEUR (discount rate = 5%/yr)")
print(f"Total trans. investment cost NS {in_year} per GW trans.: {trans_inv_cost_mEURperGW} mEUR (discount rate = 5%/yr)\n")

### HUB COST
df_hub_costs_cap = pd.read_excel("../EMPIRE_extension/EMPIRE_input/Transmission.xlsx", sheet_name="OffshoreConverterCapitalCost", header=2)\
                                    [["Period", "OffshoreConverterCapitalCost"]]
df_hub_costs_om = pd.read_excel("../EMPIRE_extension/EMPIRE_input/Transmission.xlsx", sheet_name="OffshoreConverterOMCost", header=2)\
                                    [["Period", "OffshoreConverterOMCost"]]
discount_rate = 0.05

df_hub_costs_cap["OffshoreConverterCapitalCost EUR per MW discounted"] = df_hub_costs_cap.apply(lambda row: \
                                row["OffshoreConverterCapitalCost"] / ((1+discount_rate)**(row["Period"]-1)), axis=1)
df_hub_costs_cap = df_hub_costs_cap.drop(columns=["OffshoreConverterCapitalCost"])

df_hub_costs_om["OffshoreConverterOMCost EUR per MW discounted"] = df_hub_costs_om.apply(lambda row: \
                                row["OffshoreConverterOMCost"] / ((1+discount_rate)**(row["Period"]-1)), axis=1)
df_hub_costs_om = df_hub_costs_om.drop(columns=["OffshoreConverterOMCost"])

df_hub_costs_cap["Period"] =df_hub_costs_cap["Period"].apply(lambda p: PERIODS[p-1])
df_hub_costs_om["Period"] =df_hub_costs_om["Period"].apply(lambda p: PERIODS[p-1])

df_hubs = DF_HUBS_BY_CASES[FILTER_CASE]
df_hubs = df_hubs[~df_hubs["Period"].isin(excl_periods)]
df_hubs = df_hubs.merge(df_hub_costs_cap, on="Period", how="left")
df_hubs = df_hubs.merge(df_hub_costs_om, on="Period", how="left")
df_hubs["DiscountedInvestmentCost_Euro"] = df_hubs["Converter invested capacity [MW]"] * (df_hubs["OffshoreConverterCapitalCost EUR per MW discounted"] + df_hubs["OffshoreConverterOMCost EUR per MW discounted"])

total_inst_hub_cap_GW = round(df_hubs[df_hubs["Period"] == FILTER_PERIOD]["Converter total capacity [MW]"].sum() / 1000, 1)
total_hub_inv_cost_mEUR = round(df_hubs["DiscountedInvestmentCost_Euro"].sum() / 10**6, 1)
hub_inv_cost_mEURperGW = round(total_hub_inv_cost_mEUR/total_inst_hub_cap_GW, 1) if total_inst_hub_cap_GW > 0 else 0

print(f"Total installed hub. capacity NS {in_year}: {total_inst_hub_cap_GW}GW")
print(f"Total hub investment cost NS {in_year}: {total_hub_inv_cost_mEUR} mEUR (discount rate = 5%/yr)")
print(f"Total hub investment cost NS {in_year} per GW capacity: {hub_inv_cost_mEURperGW} mEUR (discount rate = 5%/yr)\n")

### TOTAL COST 
total_inv_cost_for_NS_mEUR = round(total_hub_inv_cost_mEUR + total_trans_inv_cost_mEUR + total_inv_cost_mEUR, 1)
total_inv_cost_per_GW_inst_mEUR = round(total_inv_cost_for_NS_mEUR/total_inst_cap_GW, 1)

print(f"Total installed OW capacity NS {in_year}: {total_inst_cap_GW}GW")
print(f"Total investment cost for NS {in_year}: {total_inv_cost_for_NS_mEUR} mEUR (discount rate = 5%/yr)")
print(f"Total investment cost for installed OW in NS {in_year} per GW (incl. trans and hubs): {total_inv_cost_per_GW_inst_mEUR} mEUR (discount rate = 5%/yr)\n")

Total installed capacity NS 2050: 223.3GW
Total investment cost NS 2050: 263853.3 mEUR (discount rate = 5%/yr)
Total investment cost NS 2050 per GW installed: 1181.6 mEUR (discount rate = 5%/yr)

Total installed trans. capacity NS 2050: 409.4GW
Total trans. investment cost NS 2050: 71355.1 mEUR (discount rate = 5%/yr)
Total trans. investment cost NS 2050 per GW trans.: 174.3 mEUR (discount rate = 5%/yr)

Total installed hub. capacity NS 2050: 187.5GW
Total hub investment cost NS 2050: 31744.2 mEUR (discount rate = 5%/yr)
Total hub investment cost NS 2050 per GW capacity: 169.3 mEUR (discount rate = 5%/yr)

Total installed OW capacity NS 2050: 223.3GW
Total investment cost for NS 2050: 366952.6 mEUR (discount rate = 5%/yr)
Total investment cost for installed OW in NS 2050 per GW (incl. trans and hubs): 1643.3 mEUR (discount rate = 5%/yr)

